In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests # Getting Webpage content
from bs4 import BeautifulSoup as bs # Scraping webpages
import json
import nltk
nltk.download()
from collections import Counter
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


/Users/rishidhaka/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#fuzzy data matching
#Performance can be improved with clustering before implementation of this algo (Can also look into a match_score matrix)

#full foundation list
full_foundation_list = pd.read_csv('UT Full Foundation List .csv')
full_foundation_list.head()

full_foundation_list=full_foundation_list.astype(str)

#downloading 2020 index
# Using requests module for downloading webpage content
response2020 = requests.get('https://s3.amazonaws.com/irs-form-990/index_2020.json')
# Getting status of the request
# 200 status code means our request was successful
# 404 status code means that the resource you were looking for was not found
response2020.status_code

# Parsing html data using BeautifulSoup
soup2020 = bs(response2020.content, 'html.parser')

#Saving 990 index files as txt files for 990 returns 2020

f=open('index_2020.txt', 'w')
f.write(str(soup2020))
f.close()

#Saving summaries of 990 files (from index files) 2020
data_2020=''
summary_2020=''
with open('index_2020.txt') as json_file: 
    data_2020 = json.load(json_file)
summary_2020 = data_2020['Filings2020']
#print(len(summary_2020))

index = dict()

for entry in data_2020['Filings2020']:
  name = entry['OrganizationName'].lower()
  index[name] = entry

matched = dict()
unmatched = []

for row in full_foundation_list['NAME']:
  row = row.lower()
  keys = index.keys()
  if row in keys:
    matched[row] = index[row]
  else:
    unmatched.append(row)

index_names = list(index.keys())
#print(len(index_names))

index_dict = {}
for name in index_names:
  index_dict[name] = index[name]['URL']


#creating stopword list


index_names = list(index.keys())
listToStr_unmatched = ' '.join(map(str, unmatched))
listToStr_index = ' '.join(map(str, index_names))

listToStr = listToStr_unmatched + ' ' + listToStr_index

wordlist = listToStr.split()

wordfreq = Counter(wordlist).most_common(100)
#print wordfreq.most_common(20)

index_wo_stopwords_dict = {}

#cleaning stopwords
def clean_stopword(txt):
  stop_words = set(stopwords.words('english'))
  extended_stopwords = set([key for key, _ in wordfreq])
  updated_stop_words = stop_words | extended_stopwords
  temp_list = txt.split(" ")
  temp_list = [i for i in temp_list if i not in updated_stop_words]
  return " ".join(temp_list)

#fuzzy matching
def match_name(name, list_names, min_score=0):

  #if there are no matches, return -1
  max_score = -1    
  #returning empty name if no match
  max_name=''
  for name2 in list_names:
    score = fuzz.token_set_ratio(name,name2)
    if (score>min_score) and (score>max_score):
      max_name = name2
      max_score = score
  return (max_name, max_score)

ut_list_names = unmatched
index_names = list(index.keys())
for i in range(len(ut_list_names)):
  ut_list_names[i] = clean_stopword(ut_list_names[i])

index_names=[]
for key in index_dict.keys():
  clean_key = clean_stopword(key)
  index_names.append(clean_key)
  index_wo_stopwords_dict[clean_key] = index_dict[key]

dict_list = []

merge_table = pd.DataFrame()
flag=1

for name2search in ut_list_names:
  match = match_name(name2search, index_names, 75)

  if(flag%100==0):
    print(flag)
  flag+=1
  dict_ = {}
  dict_.update({"company name" : name2search})
  dict_.update({"match_name" : match[0]})
  dict_.update({"score" : match[1]})
  dict_.update({"URL" : index_wo_stopwords_dict[match[0]]})
  dict_list.append(dict_)

merge_table = merge_table.append(dict_list, ignore_index = True)
merge_table.to_csv('merge_table.csv', sep='\t', encoding='utf-8')

'\n#creating stopword list\n\n\nindex_names = list(index.keys())\nlistToStr_unmatched = \' \'.join(map(str, unmatched))\nlistToStr_index = \' \'.join(map(str, index_names))\n\nlistToStr = listToStr_unmatched + \' \' + listToStr_index\n\nwordlist = listToStr.split()\n\nwordfreq = Counter(wordlist).most_common(100)\n#print wordfreq.most_common(20)\n\nindex_wo_stopwords_dict = {}\n\n#cleaning stopwords\ndef clean_stopword(txt):\n  stop_words = set(stopwords.words(\'english\'))\n  extended_stopwords = set([key for key, _ in wordfreq])\n  updated_stop_words = stop_words | extended_stopwords\n  temp_list = txt.split(" ")\n  temp_list = [i for i in temp_list if i not in updated_stop_words]\n  return " ".join(temp_list)\n\n#fuzzy matching\ndef match_name(name, list_names, min_score=0):\n\n  #if there are no matches, return -1\n  max_score = -1    \n  #returning empty name if no match\n  max_name=\'\'\n  for name2 in list_names:\n    score = fuzz.token_set_ratio(name,name2)\n    if (score>min_s

In [2]:
fuzzy_df = pd.read_csv('merge_table.csv', sep='\\t')
fuzzy_df.head()

/Users/rishidhaka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,"""",company name,match_name,score,"URL"""
0,"""0",12th man,crosses 12th man,100.0,https://s3.amazonaws.com/irs-form-990/20202153...
1,"""1",2a,geshe tsulga non-profit sera je 2a monks affil,100.0,https://s3.amazonaws.com/irs-form-990/20203231...
2,"""2",4girls inc.,girls 4 science,80.0,https://s3.amazonaws.com/irs-form-990/20191315...
3,"""3",4girls incorporated,incorporated,100.0,https://s3.amazonaws.com/irs-form-990/20200098...
4,"""4",7th generation,generation,100.0,https://s3.amazonaws.com/irs-form-990/20204136...


In [3]:
perfect_match_name = []
perfect_match_url = []

for i in range(len(name1)):
  if (name1[i] == name2[i]):
    perfect_match_name.append(name1[i])
    perfect_match_url.append(url[i])

print(len(perfect_match_url))

0


In [23]:
pd_matched = pd.read_csv('merge_table - merge_table.csv', sep=',')
pd_matched.head(20)

,S.No.,company name,match_name,score,URL,Match
0,0,12th man,crosses 12th man,100.0,https://s3.amazonaws.com/irs-form-990/20202153...,T
1,1,2a,geshe tsulga non-profit sera je 2a monks affil,100.0,https://s3.amazonaws.com/irs-form-990/20203231...,T
2,2,4girls inc.,girls 4 science,80.0,https://s3.amazonaws.com/irs-form-990/20191315...,NaN
3,3,4girls incorporated,incorporated,100.0,https://s3.amazonaws.com/irs-form-990/20200098...,F
4,4,7th generation,generation,100.0,https://s3.amazonaws.com/irs-form-990/20204136...,F
5,5,80-20,20,100.0,https://s3.amazonaws.com/irs-form-990/20201163...,F
6,6,contemporary art art pace,pace,100.0,https://s3.amazonaws.com/irs-form-990/20203192...,F
7,7,g,p g,100.0,https://s3.amazonaws.com/irs-form-990/20203020...,F
8,8,a. james & alice b. clark,clark,100.0,https://s3.amazonaws.com/irs-form-990/20190318...,F
9,9,a. w. riterfamily,w,100.0,https://s3.amazonaws.com/irs-form-990/20190319...,F


In [25]:
full_foundation_list_220_2 = {}
k=0
for i in range(len(pd_matched['URL'])):
    if(pd_matched['Match'][i]=='T'):
        response = requests.get(pd_matched['URL'][i])
        full_foundation_list_220_2[k] = response.text
        k+=1

In [26]:
with open('full_foundation_list_2020_2.json', 'w') as fp:
    json.dump(full_foundation_list_220_2, fp, indent = 4)

In [24]:
url_2020 = []
for i in range(len(pd_matched['URL'])):
    if(pd_matched['Match'][i]=='T' or pd_matched['Match'][i]=='t'):
        url_2020.append(pd_matched['URL'][i])

In [126]:
#Creating csvs
#saving reqd variables

#990 fields
name = []
ein = []
url_990 = []
assets_boy = []
assets_eoy = []
net_boy = []
net_eoy = []
endwmnt_group = []
endwmnt_amt_1 = []
endwmnt_amt_2 = []
endwmnt_amt_3 = []
endwmnt_amt_4 = []

#990PF fields
name_pf = []
ein_pf = []
url_990_pf = []
assets_boy_pf = []
assets_eoy_pf = []
net_boy_pf = []
net_eoy_pf = []
grant_paid = []
grant_approved = []
invest_return = []
cash_held = []

for i in matched_990_pf:
    response = requests.get(i)
    temp = bs(response.content, 'xml')
    form_type = temp.find('ReturnTypeCd').text
    clean_name = temp.find('BusinessName').text.strip()
    clean_ein = temp.find('EIN').text
    if form_type == "990PF":
        name_pf.append(clean_name)
        ein_pf.append(clean_ein)
        url_990.append(i)
        if temp.find('TotalAssetsBOYAmt'):
            assets_boy_pf.append(temp.find('TotalAssetsBOYAmt').text)
        else:
            assets_boy_pf.append('NaN')
        if temp.find('TotalAssetsEOYAmt'): 
            assets_eoy_pf.append(temp.find('TotalAssetsEOYAmt').text)
        else:
            assets_eoy_pf.append('NaN')
        if temp.find('TotNetAstOrFundBalancesBOYAmt'):
            net_boy_pf.append(temp.find('TotNetAstOrFundBalancesBOYAmt').text)
        else:
            net_boy_pf.append('NaN')
        if temp.find('TotNetAstOrFundBalancesEOYAmt'):
            net_eoy_pf.append(temp.find('TotNetAstOrFundBalancesEOYAmt').text)
        else:
            net_eoy_pf.append('NaN')
        if temp.find('TotalGrantOrContriPdDurYrAmt'):
            grant_paid.append(temp.find('TotalGrantOrContriPdDurYrAmt').text)
        else:
            grant_paid.append('NaN')
        if temp.find('TotalGrantOrContriApprvFutAmt'):
            grant_approved.append(temp.find('TotalGrantOrContriApprvFutAmt').text)
        else:
            grant_approved.append('NaN')
        if temp.find('MinimumInvestmentReturnAmt'):
            invest_return.append(temp.find('MinimumInvestmentReturnAmt').text)
        else:
            invest_return.append('NaN')
        if temp.find('CashDeemedCharitableAmt'):
            cash_held.append(temp.find('CashDeemedCharitableAmt').text)
        else:
            cash_held.append('NaN')
    elif form_type == "990":
        name.append(clean_name)
        ein.append(clean_ein)
        url_990_pf.append(i)
        if temp.find('TotalAssetsBOYAmt'):
            assets_boy.append(temp.find('TotalAssetsBOYAmt').text)
        else:
            assets_boy.append('NaN')
        if temp.find('TotalAssetsEOYAmt'): 
            assets_eoy.append(temp.find('TotalAssetsEOYAmt').text)
        else:
            assets_eoy.append('NaN')
        if temp.find('NetAssetsOrFundBalancesBOYAmt'):
            net_boy.append(temp.find('NetAssetsOrFundBalancesBOYAmt').text)
        else:
            net_boy.append('NaN')
        if temp.find('NetAssetsOrFundBalancesEOYAmt'):
            net_eoy.append(temp.find('NetAssetsOrFundBalancesEOYAmt').text)
        else:
            net_eoy.append('NaN')
        if temp.find('CYEndwmtFundGrp'):
            endwmnt_group.append(temp.find('CYEndwmtFundGrp').text)
        else:
            endwmnt_group.append('NaN')
        if temp.find('CYMinus1YrEndwmtFundGrp'):
            endwmnt_amt_1.append(temp.find('CYMinus1YrEndwmtFundGrp').text)
        else:
            endwmnt_amt_1.append('NaN')
        if temp.find('CYMinus2YrEndwmtFundGrp'):
            endwmnt_amt_2.append(temp.find('CYMinus2YrEndwmtFundGrp').text)
        else:
            endwmnt_amt_2.append('NaN')
        if temp.find('CYMinus3YrEndwmtFundGrp'):
            endwmnt_amt_3.append(temp.find('CYMinus3YrEndwmtFundGrp').text)
        else:
            endwmnt_amt_3.append('NaN')
        if temp.find('CYMinus4YrEndwmtFundGrp'):
            endwmnt_amt_4.append(temp.find('CYMinus4YrEndwmtFundGrp').text)
        else:
            endwmnt_amt_4.append('NaN')

In [127]:
dict_990 = {}
dict_990_list = []
dict_990_pf = {}

for i in range(len(name)):
    dict_990 = {}
    dict_990.update({"Company Name" : name[i]})
    dict_990.update({"EIN" : ein[i]})
    dict_990.update({"URL" : url_990_pf[i]})
    dict_990.update({"Total Assets BOY" : assets_boy[i]})
    dict_990.update({"Total Assets EOY" : assets_eoy[i]})
    dict_990.update({"Net Assets or Fund Balances BOY" : net_boy[i]})
    dict_990.update({"Net Assets or Fund Balances EOY" : net_eoy[i]})
    dict_990.update({"Endowment group" : endwmnt_group[i]})
    dict_990.update({"Endowment amount Current Year - 1 " : endwmnt_amt_1[i]})
    dict_990.update({"Endowment amount Current Year - 2 " : endwmnt_amt_2[i]})
    dict_990.update({"Endowment amount Current Year - 3 " : endwmnt_amt_3[i]})
    dict_990.update({"Endowment amount Current Year - 4 " : endwmnt_amt_4[i]})
    dict_990_list.append(dict_990)


#print(dict_990_list)
IRS_990 = pd.DataFrame()
IRS_990 = IRS_990.append(dict_990_list, ignore_index=True)
IRS_990.to_csv('IRS_990_2018_2.csv', sep=',', encoding='utf-8')

testA = pd.read_csv('IRS_990_2018_2.csv', sep=',')
testA.head()

,Unnamed: 0,Company Name,EIN,URL,Total Assets BOY,Total Assets EOY,Net Assets or Fund Balances BOY,Net Assets or Fund Balances EOY,Endowment group,Endowment amount Current Year - 1,Endowment amount Current Year - 2,Endowment amount Current Year - 3,Endowment amount Current Year - 4
0,0,CANCER RESEARCH FOUNDATION,362385213,https://s3.amazonaws.com/irs-form-990/20182290...,9094013,9671673,8766127,9417393,\n1792708\n251072\n15129\n2028651\n,\n1679504\n176791\n50000\n13588\n1792708\n,\n1786011\n-42783\n50000\n13724\n1679504\n,\n1623846\n225514\n50000\n13349\n1786011\n,\n1520531\n216367\n100000\n13052\n1623846\n
1,1,SEG FOUNDATION,731329755,https://s3.amazonaws.com/irs-form-990/20183253...,22316336,23028294,21911674,22510000,\n15273948\n101240\n2370119\n434722\n48805\n17...,\n14316563\n332051\n899422\n385845\n111757\n15...,\n15271309\n317062\n-428746\n721375\n121687\n1...,\n15275650\n265795\n517136\n698063\n89209\n152...,\n12609407\n1063059\n2458286\n289358\n565744\n...


In [128]:
dict_990 = {}
dict_990_pf_list = []
dict_990_pf = {}

for i in range(len(name_pf)):
    dict_990_pf = {}
    dict_990_pf.update({"Company Name" : name_pf[i]})
    dict_990_pf.update({"EIN" : ein_pf[i]})
    dict_990_pf.update({"URL" : url_990[i]})
    dict_990_pf.update({"Total Assets BOY" : assets_boy_pf[i]})
    dict_990_pf.update({"Total Assets EOY" : assets_eoy_pf[i]})
    dict_990_pf.update({"Net Assets or Fund Balances BOY" : net_boy_pf[i]})
    dict_990_pf.update({"Net Assets or Fund Balances EOY" : net_eoy_pf[i]})
    dict_990_pf.update({"Grants paid during the year" : grant_paid[i]})
    dict_990_pf.update({"Grants approved for the year" : grant_approved[i]})
    dict_990_pf.update({"Minimum Investment Return Amount" : invest_return[i]})
    dict_990_pf.update({"Cash Deemed Charitable Amount" : cash_held[i]})
    dict_990_pf_list.append(dict_990_pf)

IRS_990_pf = pd.DataFrame()
IRS_990_pf = IRS_990_pf.append(dict_990_pf_list, ignore_index=True)
IRS_990_pf.to_csv('IRS_990_pf_2018_2.csv', sep=',', encoding='utf-8')

testB = pd.read_csv('IRS_990_pf_2018_2.csv', sep=',')
testB.head()

,Unnamed: 0,Company Name,EIN,URL,Total Assets BOY,Total Assets EOY,Net Assets or Fund Balances BOY,Net Assets or Fund Balances EOY,Grants paid during the year,Grants approved for the year,Minimum Investment Return Amount,Cash Deemed Charitable Amount
0,0,AUGUST MEDICAL FOUNDATION INC,464180513,https://s3.amazonaws.com/irs-form-990/20183130...,0,0,0,0,16000.0,0.0,0,0
1,1,CHARLOTTE BOSWELL HEALTH FOUNDATION,826263009,https://s3.amazonaws.com/irs-form-990/20180311...,0,883768,0,883768,45000.0,0.0,46597,14192
2,2,CASTEN FAMILY FOUNDATION INC,134074089,https://s3.amazonaws.com/irs-form-990/20183319...,3769458,3544710,3769458,3544710,372036.0,0.0,188366,57370
3,3,JANE AND LARRY COMER FOUNDATION,581413108,https://s3.amazonaws.com/irs-form-990/20182135...,603268,565989,603268,565989,35200.0,0.0,29515,8989
4,4,DON A SANDERS FAMILY FOUNDATION,760537031,https://s3.amazonaws.com/irs-form-990/20184031...,9521,70607,9521,69307,480453.0,NaN,0,0


In [37]:
#finding perfect name matches between UT full foundation list and 2020 index 

#full foundation list
full_foundation_list = pd.read_csv('UT Full Foundation List .csv')
full_foundation_list.head()

full_foundation_list=full_foundation_list.astype(str)

#downloading 2020 index
# Using requests module for downloading webpage content
response2020 = requests.get('https://s3.amazonaws.com/irs-form-990/index_2020.json')
# Getting status of the request
# 200 status code means our request was successful
# 404 status code means that the resource you were looking for was not found
response2020.status_code

# Parsing html data using BeautifulSoup
soup2020 = bs(response2020.content, 'html.parser')

#Saving 990 index files as txt files for 990 returns 2020

f=open('index_2020.txt', 'w')
f.write(str(soup2020))
f.close()

#Saving summaries of 990 files (from index files) 2020
data_2020=''
summary_2020=''
with open('index_2020.txt') as json_file: 
    data_2020 = json.load(json_file)
summary_2020 = data_2020['Filings2020']
#print(len(summary_2020))

index = dict()

for entry in data_2020['Filings2020']:
  name = entry['OrganizationName'].lower()
  index[name] = entry

matched = dict()
unmatched = []

for row in full_foundation_list['NAME']:
  row = row.lower()
  keys = index.keys()
  if row in keys:
    matched[row] = index[row]
  else:
    unmatched.append(row)

In [121]:
#downloading 2019/18 index and matching with matched names from 2020 dataset

# Using requests module for downloading webpage content
response2019 = requests.get('https://s3.amazonaws.com/irs-form-990/index_2018.json')
# Getting status of the request
# 200 status code means our request was successful
# 404 status code means that the resource you were looking for was not found
response2019.status_code

# Parsing html data using BeautifulSoup
soup2019 = bs(response2019.content, 'html.parser')

#Saving 990 index files as txt files for 990 returns 2019

f=open('index_2019.txt', 'w')
f.write(str(soup2019))
f.close()

#Saving summaries of 990 files (from index files) 2019
data_2019=''
summary_2019=''
with open('index_2019.txt') as json_file: 
    data_2019 = json.load(json_file)
summary_2019 = data_2019['Filings2018']
#print(len(summary_2019))

IRS_990 = pd.read_csv('IRS_990_2020.csv')
IRS_990_pf = pd.read_csv('IRS_990_pf_2020.csv')

index = dict()

for entry in data_2019['Filings2018']:
    name = entry['OrganizationName'].lower()
    index[name] = entry

matched_990 = []
for row in IRS_990['Company Name']:
    row=row.lower()
    keys = index.keys()
    if row in keys:
        matched_990.append(index[row]["URL"])

matched_990_pf = []
for row in IRS_990_pf['Company Name']:
    row=row.lower()
    keys = index.keys()
    if row in keys:
        matched_990_pf.append(index[row]['URL'])